# Issue 132 Notebook Toaletter nära SAT
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_132_Notebook_Toaletter_n%C3%A4ra_SAT.ipynb)
* [Issue 132](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/132)

Se liknande lösning för Roslagsleden
* nu har vi SAT = wikidata [Q131318799](https://www.wikidata.org/wiki/Q131318799)
* "leden" sitter inte ihop utan varje ö har sitt segment

Jmf dricksvatten [Issue 139](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139) 


Output 
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_28_17_12.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_30_04_22.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_07_30_04_22.html)
* [kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_2_toaletter_nara_stockholm_archipelago_trail_2025_07_30_07_16.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_17_19_29.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_16_01.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_16_01.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_23_14.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_19_23_14.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_26_15_55.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_26_15_55.html)
* [kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_27_16_53.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_27_16_53.html)




version 0.1
* unary_union --> union_all
* start updating OSM with pictures from https://commons.wikimedia.org/wiki/Category:SAT_Todo
* also count number of seats ie. toilets:number
* Fetch SAT etapper via Wikidata
* Fetch geometries per relation via Overpass (no zip-mismatch)
* Buffer 200 m (Shapely ≥2 with union_all())
* Fetch toilets as nodes/ways/relations via nwr[...] out center;
* Count toilets via toilets:number (fallback male/female/unisex → else 1)
* Join to nearest etapp, build summary (CSV)
* Build a Folium map with etapper, 200 m buffer, and toilet markers
* Export toilets within 200 m as GeoJSON + CSV and save the map HTML

version 0.2
* added mapillary
* added toilets:paper_supplied
* getting error fetching OSM with overpass moved it to one call instead one per segment

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-08-27 16:53:47


In [2]:
# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os, re, requests, html, time, random, json 
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, MultiLineString, Point, mapping
from shapely.ops import linemerge, unary_union
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup
from collections import defaultdict, deque
from datetime import datetime

# =========================
# 1) Hämta SAT-etapper via Wikidata
# =========================
print("🔍 Hämtar SAT-etapper från Wikidata...")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "id": r["osmid"]["value"],
        "label": r.get("itemLabel", {}).get("value", ""),
        "island": r.get("islandLabel", {}).get("value", "")
    })
osm_ids = [e['id'] for e in etapper]
print(f"✅ Hittade {len(osm_ids)} etapper med OSM-relationer")

# =========================
# 2) Overpass helpers
# =========================
OVERPASS_ENDPOINTS = [
    "https://overpass.kumi.systems/api/interpreter",
    "https://overpass-api.de/api/interpreter",
]

def overpass_data_age_info(data):
    try:
        ts = data.get("osm3s", {}).get("timestamp_osm_base")
        if ts:
            print(f"🕒 Overpass-databasens timestamp: {ts}")
    except Exception:
        pass

def _sleep_backoff(i):
    wait = (2 ** i) + random.random()
    time.sleep(wait)

def _overpass_post_json(query, endpoints=None, retries=3, timeout=120):
    """
    Try endpoints in order; return (data_dict, endpoint_used, error_msg).
    """
    endpoints = endpoints or OVERPASS_ENDPOINTS
    last_err = None
    for ep in endpoints:
        for i in range(retries):
            try:
                r = requests.post(ep, data={"data": query}, timeout=timeout)
                if r.status_code == 200 and "json" in (r.headers.get("Content-Type","").lower()):
                    try:
                        return r.json(), ep, None
                    except Exception as e:
                        last_err = f"JSON decode error: {e}"
                else:
                    head = (r.text or "")[:300].replace("\n", " ")
                    last_err = f"Unexpected response {r.status_code} {r.headers.get('Content-Type')} — {head}"
            except Exception as e:
                last_err = f"{type(e).__name__}: {e}"
            if i < retries - 1:
                _sleep_backoff(i)
        # try next endpoint
    return None, None, last_err

def _pick_endpoint():
    test_q = "[out:json][timeout:25];node(0,0,0,0);out;"
    data, ep, err = _overpass_post_json(test_q, retries=1, timeout=25)
    return ep or OVERPASS_ENDPOINTS[0]

overpass_url = _pick_endpoint()
print(f"🌐 Overpass endpoint: {overpass_url}")

def _chunks(iterable, n):
    it = iter(iterable)
    while True:
        batch = []
        try:
            for _ in range(n):
                batch.append(next(it))
        except StopIteration:
            pass
        if not batch:
            break
        yield batch

# =========================
# 2a) Hämta geometrier per relation (rekursivt)
# =========================
print("📡 Hämtar geometrier från Overpass (inkl. nästlade relationer) ...")

geom_per_rel = {}   # {rel_id: [LineString, ...]}
geom_per_rel_meta = defaultdict(lambda: {"mapillary_urls": set()})
all_lines = []

# Pre-init relations
for rid in osm_ids:
    geom_per_rel.setdefault(int(rid), [])

CHUNK_SIZE = 12  # minska till 6 eller 4 om timeouts

def _mapillary_urls_from_tags(tags):
    raw = (tags or {}).get("mapillary", "")
    if not raw or not str(raw).strip():
        return []
    keys = [k for k in re.split(r"[;,\s]+", str(raw).strip()) if k]
    return [f"https://www.mapillary.com/app/?pKey={k}" for k in keys]

for batch_ids in _chunks([int(x) for x in osm_ids], CHUNK_SIZE):
    ids_str = ",".join(map(str, batch_ids))

    # ⚠️ IMPORTANT: no comments; split 'way.ALL; out geom tags;'
    q = f"""
[out:json][timeout:180];
rel(id:{ids_str})->.R;
.R out body;
(.R; >>;)->.ALL;
way.ALL;
out geom tags;
""".strip()

    # (Optional) print query so you can run it in Overpass Turbo / curl
    print("\n--- OVERPASS QUERY (batch) ---\n")
    print(q)
    print("\n--- END QUERY ---\n")

    data, used_ep, err = _overpass_post_json(q, endpoints=[overpass_url] + [e for e in OVERPASS_ENDPOINTS if e != overpass_url], retries=3, timeout=180)
    if not isinstance(data, dict):
        print(f"❌ Misslyckades att hämta batch: {batch_ids} — {err}")
        continue
    if used_ep and used_ep != overpass_url:
        overpass_url = used_ep  # switch for subsequent calls

    elements = data.get("elements", []) or []
    rel_elems = [e for e in elements if e.get("type") == "relation"]
    way_elems = [e for e in elements if e.get("type") == "way"]

    rel_children = defaultdict(set)
    rel_direct_ways = defaultdict(set)

    for rel in rel_elems:
        rid = rel.get("id")
        for m in rel.get("members", []) or []:
            if m.get("type") == "relation":
                rel_children[rid].add(m.get("ref"))
            elif m.get("type") == "way":
                rel_direct_ways[rid].add(m.get("ref"))

    way_geom = {}
    way_mapillary = defaultdict(list)
    for w in way_elems:
        if "geometry" not in w:
            continue
        coords = [(pt["lon"], pt["lat"]) for pt in w["geometry"] if "lon" in pt and "lat" in pt]
        if len(coords) < 2:
            continue
        try:
            line = LineString(coords)
        except Exception:
            continue
        way_geom[w["id"]] = line
        for url in _mapillary_urls_from_tags(w.get("tags", {})):
            way_mapillary[w["id"]].append(url)

    # Collect ALL descendant ways via BFS
    for top_rel in batch_ids:
        top_rel = int(top_rel)
        visited = set()
        stack = deque([top_rel])
        all_way_ids = set()

        while stack:
            current = stack.popleft()
            if current in visited:
                continue
            visited.add(current)
            all_way_ids |= rel_direct_ways.get(current, set())
            for child in rel_children.get(current, set()):
                if child not in visited:
                    stack.append(child)

        rel_lines = []
        for wid in all_way_ids:
            geom = way_geom.get(wid)
            if geom is not None:
                rel_lines.append(geom)
                all_lines.append(geom)
                for u in way_mapillary.get(wid, []):
                    geom_per_rel_meta[top_rel]["mapillary_urls"].add(u)

        if rel_lines:
            geom_per_rel[top_rel].extend(rel_lines)

    time.sleep(1.0)  # be gentle

antal_rel = len(geom_per_rel)
antal_linjer = len(all_lines)
print(f"✅ Klart. Relationer: {antal_rel} | Linjesegment totalt: {antal_linjer}")
if antal_linjer == 0:
    print("⚠️ Overpass svarade utan way-geometrier. Sänk CHUNK_SIZE (t.ex. 6) eller kör igen senare.")

# =========================
# 2b) Bygg etapp-GeoDataFrame + samlad trail
# =========================
def _normalize_lines(lines):
    if not lines:
        return None
    try:
        merged = linemerge(unary_union(lines))
    except Exception:
        try:
            merged = linemerge(MultiLineString(lines))
        except Exception:
            return None
    gtype = getattr(merged, "geom_type", "")
    if gtype in ("LineString", "MultiLineString"):
        return merged
    if gtype == "GeometryCollection":
        only_lines = [g for g in merged.geoms if g.geom_type == "LineString"]
        if not only_lines:
            return None
        return only_lines[0] if len(only_lines) == 1 else MultiLineString(only_lines)
    return None

meta_rows = []
count_no_lines = 0

for et in etapper:
    rel_id = int(et["id"])
    lines = geom_per_rel.get(rel_id, []) or []
    if not lines:
        count_no_lines += 1
        continue
    norm = _normalize_lines(lines)
    if norm is None:
        count_no_lines += 1
        continue
    meta_rows.append({
        "rel_id": rel_id,
        "label": et.get("label", ""),
        "island": et.get("island", ""),
        "geometry": norm
    })

if meta_rows:
    meta_gdf = gpd.GeoDataFrame(meta_rows, geometry="geometry", crs="EPSG:4326")
else:
    meta_gdf = gpd.GeoDataFrame(
        {"rel_id": pd.Series(dtype="int64"),
         "label": pd.Series(dtype="object"),
         "island": pd.Series(dtype="object")},
        geometry=gpd.GeoSeries([], dtype="geometry", crs="EPSG:4326"),
        crs="EPSG:4326",
    )

gdf_trail = meta_gdf[["geometry"]].copy()
print(f"🧩 Etapper med geometri: {len(meta_gdf)} / {len(etapper)} (saknade/ogiltiga: {count_no_lines})")

# =========================
# 3) Buffert 200 m + ring 200–400 m
# =========================
print("🧮 Skapar 200 m-buffert och 200–400 m-ring...")
trail_utm = gdf_trail.to_crs(3006)
buffer_utm_200 = trail_utm.buffer(200)
buffer_utm_400 = trail_utm.buffer(400)

buffer_utm_200_u = buffer_utm_200.union_all() if hasattr(buffer_utm_200, "union_all") else buffer_utm_200.unary_union
buffer_utm_400_u = buffer_utm_400.union_all() if hasattr(buffer_utm_400, "union_all") else buffer_utm_400.unary_union
ring_utm = buffer_utm_400_u.difference(buffer_utm_200_u)

buffer_union = gpd.GeoSeries([buffer_utm_200_u], crs=3006).to_crs(4326).iloc[0]
ring_union   = gpd.GeoSeries([ring_utm],       crs=3006).to_crs(4326).iloc[0]

# =========================
# 4) Hämta toaletter (Overpass)
# =========================
def _parse_int(v):
    if v is None:
        return None
    m = re.search(r"\d+", str(v))
    return int(m.group()) if m else None

def toilets_count_from_tags(tags: dict) -> int:
    n = _parse_int((tags or {}).get("toilets:num_chambers"))
    if n is not None:
        return n
    n2 = _parse_int((tags or {}).get("toilets:number"))
    if n2 is not None:
        return n2
    parts = [
        _parse_int((tags or {}).get("male:toilets")),
        _parse_int((tags or {}).get("female:toilets")),
        _parse_int((tags or {}).get("unisex:toilets")),
    ]
    parts = [p for p in parts if p is not None]
    return sum(parts) if parts else 1

if gdf_trail.empty:
    print("⚠️ Ingen trail-geometri — hoppar över toalettfrågan.")
    gdf_toilets = gpd.GeoDataFrame(columns=["geometry","tags","id","osm_type","toilets_num"], crs="EPSG:4326")
else:
    bbox = gdf_trail.total_bounds  # [minx, miny, maxx, maxy]
    q_toilets = f"""
[out:json][timeout:60];
nwr["amenity"="toilets"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out center;
""".strip()

    print("\n--- OVERPASS QUERY (toilets) ---\n")
    print(q_toilets)
    print("\n--- END QUERY ---\n")

    data, _, err = _overpass_post_json(q_toilets, retries=4, timeout=120)
    if err:
        print(f"❌ Overpass (toilets) failed: {err}")
        elements = []
    else:
        elements = data.get("elements", []) if isinstance(data, dict) else []

    toilets = []
    for el in elements:
        tags = el.get("tags", {}) or {}
        typ = el.get("type")
        if typ == "node":
            lon, lat = el.get("lon"), el.get("lat")
        else:
            center = el.get("center") or {}
            lon, lat = center.get("lon"), center.get("lat")
        if lon is None or lat is None:
            continue
        toilets.append({
            "geometry": Point(lon, lat),
            "tags": tags,
            "id": el.get("id"),
            "osm_type": typ,
            "toilets_num": toilets_count_from_tags(tags),
        })

    gdf_toilets = gpd.GeoDataFrame(toilets, crs="EPSG:4326")
    print(f"✅ Hittade {len(gdf_toilets)} toalett-objekt inom bbox")

# =========================
# 5) Dela upp med avstånd (robust)
# =========================
# Union av hela leden i meter-CRS
trail_union_utm = meta_gdf.to_crs(3006).geometry.unary_union

# Beräkna avstånd i meter för alla toaletter
toilets_utm_all = gdf_toilets.to_crs(3006).copy()
toilets_utm_all["distance_m"] = toilets_utm_all.geometry.apply(lambda g: g.distance(trail_union_utm))

# Dela upp enligt avstånd (inkluderar gränsfallet exakt 200/400m)
toilets_utm_0_200 = toilets_utm_all[toilets_utm_all["distance_m"] <= 200].copy()
toilets_utm_200_400 = toilets_utm_all[(toilets_utm_all["distance_m"] > 200) & (toilets_utm_all["distance_m"] <= 400)].copy()

# Tillbaka till WGS84
in_0_200 = toilets_utm_0_200.to_crs(4326)
in_200_400 = toilets_utm_200_400.to_crs(4326)

print(f"✅ {len(in_0_200)} toalett-objekt inom/vid 200 m")
print(f"✅ {len(in_200_400)} toalett-objekt inom 200–400 m")

# =========================
# 6) Närmaste etapp per kategori
# =========================
meta_utm = meta_gdf.to_crs(3006)
joined_0_200 = gpd.sjoin_nearest(
    toilets_utm_0_200,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

joined_200_400 = gpd.sjoin_nearest(
    toilets_utm_200_400,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

joined_0_200 = gpd.sjoin_nearest(
    toilets_utm_0_200,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

joined_200_400 = gpd.sjoin_nearest(
    toilets_utm_200_400,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

# =========================
# 7) Summary för ≤200 m
# =========================
summary = (
    joined_0_200.assign(toilets_num=joined_0_200["toilets_num"].fillna(1))
    .groupby(["label", "island"], as_index=False)
    .agg(
        sites=("geometry", "count"),
        toilets_total=("toilets_num", "sum"),
        avg_distance_m=("distance_m", "mean"),
    )
    .assign(avg_toilets_per_site=lambda df: df["toilets_total"] / df["sites"])
    .sort_values(["toilets_total", "sites"], ascending=[False, False])
)
print("📊 Summary (≤200 m):")
print(summary.head(1000))

# -------------------------
# Helper: robust map center
# -------------------------
def compute_map_center(gdf_trail, gdf_toilets=None, default=(59.33, 18.06)):
    """
    Returns [lat, lon] for map center.
    Tries trail union centroid -> trail bbox -> toilets bbox -> default.
    Safe for empty geometries.
    """
    try:
        if gdf_trail is not None and not gdf_trail.empty:
            geom = gdf_trail.geometry
            union_geom = geom.union_all() if hasattr(geom, "union_all") else geom.unary_union
            if union_geom and not union_geom.is_empty:
                c = union_geom.centroid
                if c and not c.is_empty:
                    return [float(c.y), float(c.x)]
            minx, miny, maxx, maxy = gdf_trail.total_bounds
            if (maxx > minx) and (maxy > miny):
                return [float((miny + maxy) / 2.0), float((minx + maxx) / 2.0)]
    except Exception:
        pass

    try:
        if gdf_toilets is not None and not gdf_toilets.empty:
            minx, miny, maxx, maxy = gdf_toilets.total_bounds
            if (maxx > minx) and (maxy > miny):
                return [float((miny + maxy) / 2.0), float((minx + maxx) / 2.0)]
    except Exception:
        pass
    return [float(default[0]), float(default[1])]

# =========================
# 8) Spara filer + grundkarta
# =========================
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
os.makedirs("../kartor", exist_ok=True)

summary_csv = f"../kartor/sat_toaletter_summary_{timestamp}.csv"
summary.to_csv(summary_csv, index=False)

toilets_geojson_0_200 = f"../kartor/sat_toaletter_inrange_0_200_{timestamp}.geojson"
toilets_csv_0_200 = f"../kartor/sat_toaletter_inrange_0_200_{timestamp}.csv"
in_0_200[["id", "osm_type", "toilets_num", "tags", "geometry"]].to_file(toilets_geojson_0_200, driver="GeoJSON")
in_0_200.drop(columns="geometry").to_csv(toilets_csv_0_200, index=False)

toilets_geojson_200_400 = f"../kartor/sat_toaletter_inrange_200_400_{timestamp}.geojson"
toilets_csv_200_400 = f"../kartor/sat_toaletter_inrange_200_400_{timestamp}.csv"
in_200_400[["id", "osm_type", "toilets_num", "tags", "geometry"]].to_file(toilets_geojson_200_400, driver="GeoJSON")
in_200_400.drop(columns="geometry").to_csv(toilets_csv_200_400, index=False)

center_latlon = compute_map_center(gdf_trail, gdf_toilets, default=(59.33, 18.06))
m = folium.Map(location=center_latlon, zoom_start=9, control_scale=True)
print("🧭 Map center:", center_latlon)

# Etapper
#colors = [
#    "blue","green","purple","orange","darkred","cadetblue","lightgray","darkblue",
#    "darkgreen","pink","lightblue","lightgreen","gray","black","beige","lightred"
#] 
colors = [
    "#0d47a1", "#1b5e20", "#4a148c", "#e65100",
    "#b71c1c", "#006064", "#283593", "#2e7d32",
    "#6a1b9a", "#01579b", "#9e9d24", "#4e342e",
    "#37474f", "#7b1fa2", "#1565c0", "#2e7d32"
]
for i, row in meta_gdf.reset_index(drop=True).iterrows():
    color = colors[i % len(colors)]
    popup = f"<b>{html.escape(row['label'])}</b><br>Ö: {html.escape(row['island'])}"
    folium.GeoJson(
        data=mapping(row.geometry),
        name=row["label"],
        style_function=lambda x, c=color: {"color": c, "weight": 3}
    ).add_child(folium.Popup(popup, max_width=360)).add_to(m)

# Buffert-lager (200 m och 200–400 m)
folium.GeoJson(
    data=mapping(buffer_union),
    name="200 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

folium.GeoJson(
    data=mapping(ring_union),
    name="Ring 200–400 m",
    style_function=lambda x: {'fillColor': '#ffa500', 'color': '#ffa500', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

# =========================
# 9) Commons / Mapillary helpers + QA
# =========================
def _quote_commons(title: str) -> str:
    return requests.utils.quote(title, safe=':/%')

def commons_title_to_page(title: str) -> str:
    if not title:
        return None
    t = title.strip()
    low = t.lower()
    if low.startswith(("category:", "file:", "image:")):
        return f"https://commons.wikimedia.org/wiki/{_quote_commons(t)}"
    return f"https://commons.wikimedia.org/wiki/{_quote_commons('File:' + t)}"

def commons_title_to_filepath(title: str, thumb_width=400) -> str | None:
    if not title:
        return None
    t = title.strip()
    low = t.lower()
    if low.startswith("category:"):
        return None
    if not low.startswith(("file:", "image:")):
        t = "File:" + t
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{_quote_commons(t)}?width={thumb_width}"

_wikidata_image_cache = {}
def wikidata_p18_thumb(qid: str, thumb_width=400) -> str | None:
    if not qid:
        return None
    qid = qid.strip()
    if qid in _wikidata_image_cache:
        return _wikidata_image_cache[qid]
    try:
        url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
        data = requests.get(url, timeout=15).json()
        ent = data.get("entities", {}).get(qid, {})
        p18 = ent.get("claims", {}).get("P18", [])
        if not p18:
            _wikidata_image_cache[qid] = None
            return None
        filename = p18[0]["mainsnak"]["datavalue"]["value"]
        img_url = commons_title_to_filepath(filename, thumb_width=thumb_width)
        _wikidata_image_cache[qid] = img_url
        return img_url
    except Exception:
        _wikidata_image_cache[qid] = None
        return None

def best_image_url_from_tags(tags: dict) -> str | None:
    if not tags:
        return None
    if "image" in tags and str(tags["image"]).strip():
        first = str(tags["image"]).split(";")[0].strip()
        if first.lower().startswith(("http://", "https://")):
            return first
        return commons_title_to_filepath(first)
    if "wikimedia_commons" in tags and str(tags["wikimedia_commons"]).strip():
        title = str(tags["wikimedia_commons"]).strip()
        thumb = commons_title_to_filepath(title)
        if thumb:
            return thumb
    if "wikidata" in tags and str(tags["wikidata"]).strip():
        return wikidata_p18_thumb(tags["wikidata"])
    return None

def mapillary_links_from_tags(tags: dict) -> list[str]:
    if not tags or "mapillary" not in tags or not str(tags["mapillary"]).strip():
        return []
    raw = str(tags["mapillary"]).strip()
    keys = [k for k in re.split(r"[;,\s]+", raw) if k]
    return [f"https://www.mapillary.com/app/?pKey={k}" for k in keys]

BASIC_TAGS = [
    "amenity", "access", "opening_hours", "fee", "wheelchair",
    "toilets:num_chambers", "unisex", "drinking_water", "changing_table",
    "toilets:paper_supplied", "indoor", "operator", "website", "source"
]

def missing_tags(tags: dict) -> list:
    missing = []
    tags = tags or {}
    for k in BASIC_TAGS:
        if k == "amenity":
            if tags.get("amenity") != "toilets":
                missing.append("amenity=toilets")
            continue
        v = tags.get(k)
        if v is None or str(v).strip() == "":
            missing.append(k)
    if "unisex" in missing and (("male" in tags or "male:toilets" in tags) and ("female" in tags or "female:toilets" in tags)):
        try:
            missing.remove("unisex")
        except ValueError:
            pass
    return missing

def build_basic_table(joined_df):
    tag_cols = [
        "access","opening_hours","fee","wheelchair","toilets:num_chambers","unisex",
        "drinking_water","changing_table","toilets:paper_supplied","indoor","operator","website","source",
        "wikidata","wikimedia_commons","image","mapillary",
        # extra keys you mentioned
        "toilets:handwashing","toilets:position"
    ]
    rows = []
    for _, r in joined_df.to_crs(4326).iterrows():
        tags = r.get("tags", {}) or {}
        # Decode if tags arrived as a JSON string
        if isinstance(tags, str):
            try:
                tags = json.loads(tags)
            except Exception:
                tags = {}

        commons_title = str(tags.get("wikimedia_commons") or "").strip() or None
        commons_page_url = commons_title_to_page(commons_title) if commons_title else None

        img_url = best_image_url_from_tags(tags)  # uses image or Commons/Wikidata fallback
        miss = missing_tags(tags)
        mp_links = mapillary_links_from_tags(tags)

        row = {
            "id": r["id"],
            "osm_type": r["osm_type"],
            "lat": r.geometry.y,
            "lon": r.geometry.x,
            "label": r.get("label", ""),
            "island": r.get("island", ""),
            "distance_m": round(float(r.get("distance_m", 0)), 1),
            "toilets_num": int(r.get("toilets_num", 1)),
            "image_url": img_url,
            "missing_tags": ", ".join(miss),
            "wikimedia_commons_title": commons_title,
            "wikimedia_commons_url": commons_page_url,
            "mapillary_links": mp_links,
        }
        for k in tag_cols:
            row[k] = (tags.get(k) if isinstance(tags, dict) else None)
        rows.append(row)
    return pd.DataFrame(rows)

# =========================
# 10) Bygg tabeller och lager
# =========================
basic_df_0_200 = build_basic_table(joined_0_200)
basic_df_200_400 = build_basic_table(joined_200_400)

timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
os.makedirs("../kartor", exist_ok=True)

basic_csv_0_200 = f"../kartor/sat_toaletter_basic_tags_0_200_{timestamp}.csv"
basic_df_0_200.to_csv(basic_csv_0_200, index=False)
basic_csv_200_400 = f"../kartor/sat_toaletter_basic_tags_200_400_{timestamp}.csv"
basic_df_200_400.to_csv(basic_csv_200_400, index=False)

print("🧾 Basic CSV (≤200 m):", basic_csv_0_200)
print("🧾 Basic CSV (200–400 m):", basic_csv_200_400)

def add_markers(df, feature_group, marker_color):
    for _, r in df.iterrows():
        osm_url = f"https://www.openstreetmap.org/{'node' if r['osm_type']=='node' else 'way' if r['osm_type']=='way' else 'relation'}/{r['id']}"
        img_html = (f'<div style="margin:6px 0">'f'<img src="{html.escape(str(r["image_url"]))}" loading="lazy" referrerpolicy="no-referrer" '
            f'style="max-width:280px; height:auto; display:block;" />'
            f'</div>'
            ) if r.get("image_url") else ""
        miss_html = f"<div style='color:#b00; margin-top:4px'><b>Saknade taggar:</b> {html.escape(str(r['missing_tags']))}</div>" if r.get("missing_tags") else ""
        commons_html = ""
        if r.get("wikimedia_commons_url"):
            title = r.get("wikimedia_commons_title") or "Wikimedia Commons"
            commons_html = f'<div style="margin-top:4px">📷 <a href="{html.escape(str(r["wikimedia_commons_url"]))}" target="_blank">{html.escape(str(title))}</a></div>'

        mp_links = r.get("mapillary_links") or []
        if isinstance(mp_links, str):
            mp_links = [u for u in re.split(r"[;\s,]+", mp_links) if u]
        mapillary_html = ""
        if mp_links:
            items = "".join(f'<li><a href="{html.escape(u)}" target="_blank" rel="noopener">Öppna i Mapillary</a></li>' for u in mp_links[:5])
            mapillary_html = f'<div style="margin-top:4px">🗺️ Mapillary:<ul style="margin:4px 0 0 18px">{items}</ul></div>'
        kv = []
        for k in ["opening_hours","fee","wheelchair","access","toilets:num_chambers","unisex","drinking_water","changing_table","toilets:paper_supplied"]:
            v = r.get(k)
            if pd.notna(v) and str(v).strip() != "":
                kv.append(f"{k}={html.escape(str(v))}")

        kv_html = ("<div>" + "<br>".join(kv) + "</div>") if kv else ""

        popup_html = f"""
        <div style="font-size:13px; line-height:1.4">
          <b><a href="{osm_url}" target="_blank">OSM ({r['osm_type']}) {int(r['id'])}</a></b><br>
          Etapp: <b>{html.escape(str(r.get('label') or ''))}</b> (Ö: {html.escape(str(r.get('island') or ''))})<br>
          Avstånd: ~{r['distance_m']} m<br>
          Antal toaletter: <b>{int(r['toilets_num'])}</b>
          {img_html}
          {kv_html}
          {commons_html}
          {mapillary_html}
          {miss_html}
        </div>
        """
        Marker(
            location=[r["lat"], r["lon"]],
            popup=Popup(popup_html, max_width=320),
            icon=Icon(color=marker_color, icon="info-sign")
        ).add_to(feature_group)

# ≤200 m lager
toilets_pic_fg_0_200 = FeatureGroup(name="Toaletter ≤200 m (bild, Commons, Mapillary, saknade taggar)")
add_markers(basic_df_0_200, toilets_pic_fg_0_200, marker_color="darkblue")
toilets_pic_fg_0_200.add_to(m)

# 200–400 m lager
toilets_pic_fg_200_400 = FeatureGroup(name="Toaletter 200–400 m (bild, Commons, Mapillary, saknade taggar)")
add_markers(basic_df_200_400, toilets_pic_fg_200_400, marker_color="orange")
toilets_pic_fg_200_400.add_to(m)

LayerControl(collapsed=True).add_to(m)  
# =========================
# 11) Spara karta + output
# =========================
map_html = f"../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_{timestamp}.html"
m.save(map_html)

print("✅ Klart!")
print(f"• Summary CSV (≤200 m): {summary_csv}")
print(f"• Toilets GeoJSON ≤200 m: {toilets_geojson_0_200}")
print(f"• Toilets CSV ≤200 m: {toilets_csv_0_200}")
print(f"• Toilets GeoJSON 200–400 m: {toilets_geojson_200_400}")
print(f"• Toilets CSV 200–400 m: {toilets_csv_200_400}")
print(f"• Basic CSV ≤200 m: {basic_csv_0_200}")
print(f"• Basic CSV 200–400 m: {basic_csv_200_400}")
print(f"• Karta: {map_html}")


🔍 Hämtar SAT-etapper från Wikidata...
✅ Hittade 20 etapper med OSM-relationer
🌐 Overpass endpoint: https://overpass.kumi.systems/api/interpreter
📡 Hämtar geometrier från Overpass (inkl. nästlade relationer) ...

--- OVERPASS QUERY (batch) ---

[out:json][timeout:180];
rel(id:19012436,19020231,19013576,19079703,19013473,19014571,19015969,19012684,19012654,19023687,19016280,19014515)->.R;
.R out body;
(.R; >>;)->.ALL;
way.ALL;
out geom tags;

--- END QUERY ---


--- OVERPASS QUERY (batch) ---

[out:json][timeout:180];
rel(id:19141225,19013472,19080874,19018272,19020310,19023630,19016187,19081125)->.R;
.R out body;
(.R; >>;)->.ALL;
way.ALL;
out geom tags;

--- END QUERY ---

✅ Klart. Relationer: 20 | Linjesegment totalt: 625
🧩 Etapper med geometri: 20 / 20 (saknade/ogiltiga: 0)
🧮 Skapar 200 m-buffert och 200–400 m-ring...

--- OVERPASS QUERY (toilets) ---

[out:json][timeout:60];
nwr["amenity"="toilets"](58.7378793,17.8574954,59.8625492,19.1391668);
out center;

--- END QUERY ---

✅ Hitta

/var/folders/fd/md6r13sj0wsbg_6_xl160d300000gn/T/ipykernel_8471/3321270200.py:366: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  trail_union_utm = meta_gdf.to_crs(3006).geometry.unary_union


✅ Klart!
• Summary CSV (≤200 m): ../kartor/sat_toaletter_summary_2025_08_27_16_53.csv
• Toilets GeoJSON ≤200 m: ../kartor/sat_toaletter_inrange_0_200_2025_08_27_16_53.geojson
• Toilets CSV ≤200 m: ../kartor/sat_toaletter_inrange_0_200_2025_08_27_16_53.csv
• Toilets GeoJSON 200–400 m: ../kartor/sat_toaletter_inrange_200_400_2025_08_27_16_53.geojson
• Toilets CSV 200–400 m: ../kartor/sat_toaletter_inrange_200_400_2025_08_27_16_53.csv
• Basic CSV ≤200 m: ../kartor/sat_toaletter_basic_tags_0_200_2025_08_27_16_53.csv
• Basic CSV 200–400 m: ../kartor/sat_toaletter_basic_tags_200_400_2025_08_27_16_53.csv
• Karta: ../kartor/Issue_132_toaletter_nara_stockholm_archipelago_trail_2025_08_27_16_53.html


In [3]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-08-27 16:54:00
Total time elapsed: 12.28 seconds
